<a href="https://colab.research.google.com/github/dropcreations/FFmpeg-for-Google-Colab/blob/main/FFmpeg_in_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Mount Google Drive__

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="40" alt="Gdrive-logo"/></center>
#@markdown <center><h3><b>Mount Google Drive</b></h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

# __Install FFmpeg__

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/dropcreations/FFmpeg-for-Google-Colab/main/ffmpeg-logo.png' height="50" alt="FFmpeg-logo"/></center>
#@markdown <center><h3><b>Install FFmpeg</b></h3></center><br>
from IPython.display import clear_output
!sudo curl -L https://github.com/BtbN/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz -o /usr/local/bin/ffmpeg.tar.xz
clear_output()
%cd /usr/local/bin/
clear_output()
!7z e /usr/local/bin/ffmpeg.tar.xz
clear_output()
!7z e /usr/local/bin/ffmpeg.tar
clear_output()
!sudo chmod a+rx /usr/local/bin/ffmpeg
clear_output()
%cd /content/
!sudo curl -L https://mkvtoolnix.download/appimage/MKVToolNix_GUI-66.0.0-x86_64.AppImage -o /usr/local/bin/MKVToolNix_GUI-66.0.0-x86_64.AppImage
!sudo chmod u+rx /usr/local/bin/MKVToolNix_GUI-66.0.0-x86_64.AppImage
!sudo ln -s /usr/local/bin/MKVToolNix_GUI-66.0.0-x86_64.AppImage /usr/local/bin/mkvmerge
!sudo chmod a+rx /usr/local/bin/mkvmerge
clear_output()
print("Successfully Installed.")

# __Run FFmpeg Scripts__

### __Remux Video files__

* **Remux** video files without re-encoding.
* Add `input_file` path and select a `output format`.
* Make sure that you **selected** output format **supports** to the all track types of input file.

In [ ]:
input_file = "" #@param {type:"string"}
output_format = "mp4" #@param ["mp4", "mkv", "mov", "m4v", "webm", "other"]
save_in_source_dir = True #@param {type:"boolean"}

import os

cmd_line = '-hide_banner -i "' + input_file + '" -map 0 -c copy '

if output_format == "other":
    output_format = input("Enter output format: ").lower()

if save_in_source_dir == True:
    output_folder = os.path.dirname(input_file)
    output_filename = os.path.basename(input_file)
    output_filename_without_extension = os.path.splitext(output_filename)
    cmd_line = cmd_line + '"' + output_folder + "/" + output_filename_without_extension[0] + "." + output_format + '"'
else:
    output_folder = input("Enter output folder path: ")
    output_filename = os.path.basename(input_file)
    output_filename_without_extension = os.path.splitext(output_filename)
    cmd_line = cmd_line + '"' + output_folder + "/" + output_filename_without_extension[0] + "." + output_format + '"'

!ffmpeg {cmd_line}

### __Trim Video Files__

* **Trim** video files without re-encoding.
* Add `input_file` path and **time duration** to trim.

In [ ]:
input_file = "" #@param {type:"string"}
start_time = "00:00:00.000" #@param {type:"string"}
end_time = "00:00:00.000" #@param {type:"string"}
save_in_source_dir = True #@param {type:"boolean"}

import os

cmd_line = '-hide_banner -i "' + input_file + '" -map 0 -c copy -ss ' + start_time + ' -to ' + end_time + " "

if save_in_source_dir == True:
    output_folder = os.path.dirname(input_file)
    output_filename = os.path.basename(input_file)
    output_filename_without_extension = os.path.splitext(output_filename)
    cmd_line = cmd_line + '"' + output_folder + "/" + output_filename_without_extension[0] + "_TRIMED" + output_filename_without_extension[-1] + '"'
else:
    output_folder = input("Enter output folder path: ")
    output_filename = os.path.basename(input_file)
    output_filename_without_extension = os.path.splitext(output_filename)
    cmd_line = cmd_line + '"' + output_folder + "/" + output_filename_without_extension[0] + "_TRIMED" + output_filename_without_extension[-1] + '"'

!ffmpeg {cmd_line}

### __Extract Audio__

* **Extract** audio from video files.
* When extracting **DTS** and **TrueHD** tracks they will be mux to **.mka**

In [ ]:
input_file = "" #@param {type:"string"}
save_in_source_dir = True #@param {type:"boolean"}

import os
import json
import subprocess

ffprobe_json_out = subprocess.check_output([
    'ffprobe',
    '-hide_banner',
    '-loglevel',
    'warning',
    '-print_format',
    'json',
    '-show_entries',
    'stream',
    '-i',
    input_file,
    ], stderr=subprocess.DEVNULL)
json_data = json.loads(ffprobe_json_out)

stream_count = len(json_data['streams'])
cmd_line = '-hide_banner -i "' + input_file + '" '

for i in range(stream_count):
    codec_type = json_data.get('streams')[int(i)].get('codec_type')
    if codec_type == "audio":
        index = json_data.get('streams')[int(i)].get('index')
        codec_name = json_data.get('streams')[int(i)].get('codec_name')
        channel_layout = json_data.get('streams')[int(i)].get('channels')
        
        print(str(index) + " : " + codec_name + " : " + str(channel_layout) + " Channels")

track_id = input("Enter ID: ")
codec_name = json_data.get('streams')[int(track_id)].get('codec_name')
if "opus" in codec_name:
    codec_ext = "ogg"
elif "dts" in codec_name:
    codec_ext = "mka"
elif "aac" in codec_name:
    codec_ext = "aac"
elif "truehd" in codec_name:
    codec_ext = "mka"
elif codec_name == "ac3":
    codec_ext = "ac3"
elif codec_name == "eac3":
    codec_ext = "eac3"
elif "pcm" in codec_name:
    codec_ext = "wav"
elif "flac" in codec_name:
    codec_ext = "flac"

cmd_line = cmd_line + "-vn -sn -map 0:" + str(track_id) + " -c copy "

if save_in_source_dir == True:
    output_folder = os.path.dirname(input_file)
    output_filename = os.path.basename(input_file)
    output_filename_without_extension = os.path.splitext(output_filename)
    cmd_line = cmd_line + '"' + output_folder + "/" + output_filename_without_extension[0] + "." + codec_ext + '"'
else:
    output_folder = input("Enter output folder path: ")
    output_filename = os.path.basename(input_file)
    output_filename_without_extension = os.path.splitext(output_filename)
    cmd_line = cmd_line + '"' + output_folder + "/" + output_filename_without_extension[0] + "." + codec_ext + '"'

!ffmpeg {cmd_line}

### __Remove Bitstream Metadata__

* For a **AVC (H.264)** track run below cell.
* Output file will be saved to the **input file's folder**.

In [ ]:
input_file = "" #@param {type:"string"}
save_in_source_dir = True #@param {type:"boolean"}

import os

cmd_line = '-hide_banner -i "' + input_file + '" -map 0 -c copy -bitexact -map_metadata -1 -vbsf filter_units=remove_types=6 '

if save_in_source_dir == True:
    output_folder = os.path.dirname(input_file)
    output_filename = os.path.basename(input_file)
    output_filename_without_extension = os.path.splitext(output_filename)
    cmd_line = cmd_line + '"' + output_folder + "/" + output_filename_without_extension[0] + "_BSF_FIXED" + output_filename_without_extension[-1] + '"'
else:
    output_folder = input("Enter output folder path: ")
    output_filename = os.path.basename(input_file)
    output_filename_without_extension = os.path.splitext(output_filename)
    cmd_line = cmd_line + '"' + output_folder + "/" + output_filename_without_extension[0] + "_BSF_FIXED" + output_filename_without_extension[-1] + '"'

print(f'!ffmpeg {cmd_line}')

* For a **HEVC (H.265)** track run below cell.
* This cell will add **only the video track** to the **mkv output**.

In [ ]:
input_file = "" #@param {type:"string"}
save_in_source_dir = True #@param {type:"boolean"}

import os
from IPython.display import clear_output

cmd_line_1 = '-hide_banner -i "' + input_file + '" -an -sn -c:v copy -vbsf hevc_mp4toannexb /content/source-video.hevc'
cmd_line_2 = '-hide_banner -i /content/source-video.hevc -c copy -vbsf filter_units=remove_types=39 /content/output.hevc'

if save_in_source_dir == True:
    output_folder = os.path.dirname(input_file)
    output_filename = os.path.basename(input_file)
    output_filename_without_extension = os.path.splitext(output_filename)
    cmd_line_3 = '--output "' + output_folder + "/" + output_filename_without_extension[0] + '_BSF_FIXED.mkv" /content/output.hevc'
else:
    output_folder = input("Enter output folder path: ")
    output_filename = os.path.basename(input_file)
    output_filename_without_extension = os.path.splitext(output_filename)
    cmd_line_3 = '--output "' + output_folder + "/" + output_filename_without_extension[0] + '_BSF_FIXED.mkv" /content/output.hevc'

!ffmpeg {cmd_line_1}
clear_output()
!ffmpeg {cmd_line_2}
clear_output()
!rm /content/source-video.hevc
!mkvmerge {cmd_line_3}
!rm /content/output.hevc
clear_output()
print("Completed.")

### __Encode H.264 | CRF__

* Enter file paths and output format.
* **Change** video settings as you want.
* If you also want to **encode the audio** track, select a **codec**.

* `aac`        - Advanced Audio Codec (AAC) <br>
* `eac3`       - Dolby Digital Plus (EAC-3) <br>
* `pcm_s24le`  - Lossless WAV (24bit) <br>
* `flac`       - Lossless FLAC <br>
* `libopus`    - Opus Audio <br>

* Change **bitrate** and **channel** settings to get various audio **qualities**.

* `2` - Stereo
* `6` - Surround 5.1
* `8` - Surround 7.1

In [ ]:
Input_file = ""  # @param {type:"string"}
Output_file = ""  # @param {type:"string"}
Output_format = "mkv" #@param ["mp4", "mkv", "mov"]
#@markdown <h3><b>Video Settings</b></h3>
Preset = "veryslow" #@param ["ultrafast", "superfast", "veryfast", "faster", "fast", "medium", "slow", "slower", "veryslow", "placebo"]
Profile = "high" #@param ["baseline", "main", "high", "high10", "high422", "high444"]
Level = "4" #@param ["1", "1.1", "1.2", "1.3", "2", "2.1", "2.2", "3", "3.1","3.2", "4", "4.1", "4.2", "5", "5.1", "5.2"]
Tune = "None" #@param ["None", "film", "animation", "grain", "stillimage", "psnr", "ssim", "fastdecode", "zerolatency"]
CRF = 19 #@param {type:"slider", min:0, max:51, step:1}
#@markdown <h3><b>Audio Settings</b></h3>
Audio_codec = "copy" #@param ["copy", "aac", "eac3", "pcm_s24le", "flac", "libopus"]
Audio_bitrate = "320"  # @param {type:"string"}
Audio_channels = "2" #@param ["2", "6", "8"]

if Tune == "None":
  cmd = !ffmpeg -i "{Input_file}" -c:v libx264 -crf $CRF -preset $Preset -profile:v $Profile -level $Level -maxrate 20000k -bufsize 25000k -partitions all -me_range 120 -refs 4 -keyint_min 121 -rc-lookahead 240 -nal-hrd 1 -sc_threshold 0 -threads 8 -x264-params "colorprim=bt709:colormatrix=bt709:transfer=bt709:qpmin=6:qpmax=51:bframes=16:lookahead-threads=2:stitchable=1:keyint=240" -bitexact -map_metadata -1 -vbsf filter_units=remove_types=6 -c:a $Audio_codec -b:a {Audio_bitrate}k -ac $Audio_channels "{Output_file}.{Output_format}"
  !$cmd
else:
  cmd = !ffmpeg -i "{Input_file}" -c:v libx264 -crf $CRF -tune {Tune} -preset $Preset -profile:v $Profile -level $Level -maxrate 20000k -bufsize 25000k -partitions all -me_range 120 -refs 4 -keyint_min 121 -rc-lookahead 240 -nal-hrd 1 -sc_threshold 0 -threads 8 -x264-params "colorprim=bt709:colormatrix=bt709:transfer=bt709:qpmin=6:qpmax=51:bframes=16:lookahead-threads=2:stitchable=1:keyint=240" -bitexact -map_metadata -1 -vbsf filter_units=remove_types=6 -c:a $Audio_codec -b:a {Audio_bitrate}k -ac $Audio_channels "{Output_file}.{Output_format}"
  !$cmd

### __Encode H.264 | 2-Pass__

* Enter file paths and output format.
* **Change** video settings as you want.
* If you also want to **encode the audio** track, select a **codec**.
* **X** = Bitrate **/ 2** ,<br>
  **Maxrate** = Bitrate + **X**<br>
  **Bufsize** = Maxrate + **X**

* `aac`        - Advanced Audio Codec (AAC) <br>
* `eac3`       - Dolby Digital Plus (EAC-3) <br>
* `pcm_s24le`  - Lossless WAV (24bit) <br>
* `flac`       - Lossless FLAC <br>
* `libopus`    - Opus Audio <br>

* Change **bitrate** and **channel** settings to get various audio **qualities**.

* `2` - Stereo
* `6` - Surround 5.1
* `8` - Surround 7.1

In [ ]:
Input_file = ""  #@param {type:"string"}
Output_file = ""  #@param {type:"string"}
Output_format = "mkv" #@param ["mp4", "mkv", "mov"]
#@markdown <h3><b>Video Settings</b></h3>
Preset = "veryslow" #@param ["ultrafast", "superfast", "veryfast", "faster", "fast", "medium", "slow", "slower", "veryslow", "placebo"]
Profile = "high" #@param ["baseline", "main", "high", "high10", "high422", "high444"]
Level = "4" #@param ["1", "1.1", "1.2", "1.3", "2", "2.1", "2.2", "3", "3.1","3.2", "4", "4.1", "4.2", "5", "5.1", "5.2"]
Tune = "None" #@param ["None", "film", "animation", "grain", "stillimage", "psnr", "ssim", "fastdecode", "zerolatency"]
Bitrate = "5800" #@param {type:"string"}
Maxrate = "8700" #@param {type:"string"}
Bufsize = "11600" #@param {type:"string"}
#@markdown <h3><b>Audio Settings</b></h3>
Audio_codec = "copy" #@param ["copy", "aac", "eac3", "pcm_s24le", "flac", "libopus"]
Audio_bitrate = "320"  #@param {type:"string"}
Audio_channels = "2" #@param ["2", "6", "8"]

if Tune == "None":
  cmd = !ffmpeg -i "{Input_file}" -c:v libx264 -pass 1 -b:v {Bitrate}k -preset $Preset -profile:v $Profile -level $Level -maxrate {Maxrate}k -bufsize {Bufsize}k -partitions all -me_range 24 -refs 4 -keyint_min 24 -rc-lookahead 48 -nal-hrd 1 -sc_threshold 0 -threads 8 -x264-params "colorprim=bt709:colormatrix=bt709:transfer=bt709:qpmin=6:qpmax=51:bframes=4:stitchable=1:keyint=48" -an -y -f null /dev/null
  !$cmd
  cmd = !ffmpeg -i "{Input_file}" -c:v libx264 -pass 2 -b:v {Bitrate}k -preset $Preset -profile:v $Profile -level $Level -maxrate {Maxrate}k -bufsize {Bufsize}k -partitions all -me_range 24 -refs 4 -keyint_min 24 -rc-lookahead 48 -nal-hrd 1 -sc_threshold 0 -threads 8 -x264-params "colorprim=bt709:colormatrix=bt709:transfer=bt709:qpmin=6:qpmax=51:bframes=4:stitchable=1:keyint=48" -bitexact -map_metadata -1 -vbsf filter_units=remove_types=6 -c:a $Audio_codec -b:a {Audio_bitrate}k -ac $Audio_channels "{Output_file}.{Output_format}"
  !$cmd
else:
  cmd = !ffmpeg -i "{Input_file}" -c:v libx264 -pass 1 -b:v {Bitrate}k -tune {Tune} -preset $Preset -profile:v $Profile -level $Level -maxrate {Maxrate}k -bufsize {Bufsize}k -partitions all -me_range 24 -refs 4 -keyint_min 24 -rc-lookahead 48 -nal-hrd 1 -sc_threshold 0 -threads 8 -x264-params "colorprim=bt709:colormatrix=bt709:transfer=bt709:qpmin=6:qpmax=51:bframes=4:stitchable=1:keyint=48" -an -y -f null /dev/null
  !$cmd
  cmd = !ffmpeg -i "{Input_file}" -c:v libx264 -pass 2 -b:v {Bitrate}k -tune {Tune} -preset $Preset -profile:v $Profile -level $Level -maxrate {Maxrate}k -bufsize {Bufsize}k -partitions all -me_range 24 -refs 4 -keyint_min 24 -rc-lookahead 48 -nal-hrd 1 -sc_threshold 0 -threads 8 -x264-params "colorprim=bt709:colormatrix=bt709:transfer=bt709:qpmin=6:qpmax=51:bframes=4:stitchable=1:keyint=48" -bitexact -map_metadata -1 -vbsf filter_units=remove_types=6 -c:a $Audio_codec -b:a {Audio_bitrate}k -ac $Audio_channels "{Output_file}.{Output_format}"
  !$cmd

### __HDR to SDR__

* Convert **HDR** video to **SDR** from below cell.
* Enter **file path** and selcet a video **codec**.

In [ ]:
input_file = "" #@param {type:"string"}
codec = "HEVC" #@param ["H.264", "HEVC"]
crf = 22 #@param {type:"slider", min:0, max:51, step:1}
bit_depth = "10-Bit" #@param ["8-Bit", "10-Bit"]
tone_map = "hable" #@param ["none", "clip", "linear", "gamma", "reinhard", "hable", "mobius"]
save_in_source_dir = True #@param {type:"boolean"}

import os

if codec == "H.264":
  codec = "libx264"
elif codec == "HEVC":
  codec = "libx265"

if bit_depth == "8-Bit":
    bit_depth = "yuv420p"
elif bit_depth == "10-Bit":
    bit_depth = "yuv420p10le"

cmd_line = '-hide_banner -i "' + input_file + '" -max_muxing_queue_size 1024 -c:v ' + codec + " -crf:v " + str(crf) + " -preset:v medium -pix_fmt " + bit_depth + " "
if codec == "libx265":
    cmd_line = cmd_line + '-x265-params "aq-mode=2:repeat-headers=0:strong-intra-smoothing=1:bframes=4:b-adapt=2:frame-threads=0" '
cmd_line = cmd_line + "-vf zscale=t=linear:npl=100,format=gbrpf32le,zscale=p=bt709,tonemap=tonemap=" + tone_map + ":desat=0,zscale=t=bt709:m=bt709:r=tv,format=" + bit_depth + " "

if save_in_source_dir == True:
    output_folder = os.path.dirname(input_file)
    output_filename = os.path.basename(input_file)
    output_filename_without_extension = os.path.splitext(output_filename)
    cmd_line = cmd_line + '"' + output_folder + "/" + output_filename_without_extension[0] + "_SDR" + output_filename_without_extension[-1] + '"'
else:
    output_folder = input("Enter output folder path: ")
    output_filename = os.path.basename(input_file)
    output_filename_without_extension = os.path.splitext(output_filename)
    cmd_line = cmd_line + '"' + output_folder + "/" + output_filename_without_extension[0] + "_SDR" + output_filename_without_extension[-1] + '"'

print(f'!ffmpeg {cmd_line}')